In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import os

#### Functions

In [2]:
def dist(stop_lat, stop_lon,next_o_lat,next_o_lon):
    return np.arccos(
        np.sin(np.radians(stop_lat)) * np.sin(np.radians(next_o_lat)) + 
        np.cos(np.radians(stop_lat)) * np.cos(np.radians(next_o_lat)) * 
            np.cos(np.radians(stop_lon) - np.radians(next_o_lon))
    ) * 6371

def get_router_id(query_date):
    INTERMEDIATE_OTP_DATE = pd.to_datetime("2017-06-30", format="%Y-%m-%d")
    router_id = ''

    if (query_date <= INTERMEDIATE_OTP_DATE):
        return 'ctba-2017-1'
    else:
        return 'ctba-2017-2'

#### Read Origin/Next-Origin Pairs

In [3]:
trips_origins = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/enhanced-buste/2017_05_09_user_trips.csv', parse_dates=['o_boarding_datetime','o_gps_datetime','next_o_boarding_datetime','next_o_gps_datetime'])

In [4]:
len(trips_origins)

140101

In [5]:
trips_origins.sort_values(['cardNum','o_boarding_id']).head(20)

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
28795,233641.0,2,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,...,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,0 days 05:28:56.000000000,14.636225
28796,233641.0,3,2017-05-09 12:08:17,021,04020,NaN,14508,NaT,-25.441614,-49.346740,...,2017-05-09 06:39:21,000,01026,NaN,41863,NaT,-25.423639,-49.202354,0 days 05:28:56.000000000,14.636225
79789,257342.0,4,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,...,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,0 days 09:35:53.000000000,3.178960
79790,257342.0,5,2017-05-09 17:24:51,000,01021,NaN,26037,NaT,-25.471734,-49.237478,...,2017-05-09 07:48:58,511,EA172,4.0,31195,2017-05-09 07:47:51,-25.498750,-49.247840,0 days 09:35:53.000000000,3.178960
112045,304127.0,9,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,...,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,0 days 00:32:46.000000000,2.639323
112046,304127.0,10,2017-05-09 10:57:36,370,LC016,3.0,3377,2017-05-09 10:42:18,-25.428142,-49.246865,...,2017-05-09 10:24:50,175,BC010,5.0,31748,2017-05-09 10:22:28,-25.450968,-49.254072,0 days 00:32:46.000000000,2.639323
134533,304627.0,12,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,...,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,0 days 02:07:17.000000000,8.248431
134534,304627.0,13,2017-05-09 16:24:40,370,LC020,6.0,30303,2017-05-09 15:45:37,-25.455659,-49.323150,...,2017-05-09 14:17:23,370,BC032,6.0,3377,2017-05-09 14:16:09,-25.428142,-49.246865,0 days 02:07:17.000000000,8.248431
115635,305601.0,14,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,...,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,0 days 02:25:53.000000000,6.307331
115636,305601.0,15,2017-05-09 13:32:37,000,00045,NaN,43723,NaT,-25.433715,-49.270258,...,2017-05-09 11:06:44,000,05007,NaN,14517,NaT,-25.377432,-49.262446,0 days 02:25:53.000000000,6.307331


In [6]:
trips_origins.dist_between_origins.describe()

count    140101.000000
mean          6.822495
std           3.605439
min           1.500231
25%           4.005281
50%           6.368706
75%           8.972474
max          28.419669
Name: dist_between_origins, dtype: float64

In [7]:
trips_origins.boardings_timediff.describe()

count                        140101
unique                        37746
top       0 days 10:23:53.000000000
freq                             21
Name: boardings_timediff, dtype: object

In [8]:
len(trips_origins)

140101

In [9]:
trips_origins.dtypes

cardNum                            float64
o_boarding_id                        int64
o_boarding_datetime         datetime64[ns]
o_route                             object
o_busCode                           object
o_tripNum                          float64
o_stopPointId                        int64
o_gps_datetime              datetime64[ns]
o_stop_lat                         float64
o_stop_lon                         float64
next_o_boarding_id                   int64
next_o_boarding_datetime    datetime64[ns]
next_o_route                        object
next_o_busCode                      object
next_o_tripNum                     float64
next_o_stopPointId                   int64
next_o_gps_datetime         datetime64[ns]
next_o_stop_lat                    float64
next_o_stop_lon                    float64
boardings_timediff                  object
dist_between_origins               float64
dtype: object

#### Read OTP Suggestions

In [10]:
otp_suggestions = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/otp-itineraries/2017_05_09_otp_itineraries.csv', parse_dates=['date','otp_start_time','otp_end_time'])
otp_suggestions['otp_start_time'] = otp_suggestions['otp_start_time'] - pd.Timedelta('10800 s')
otp_suggestions['otp_end_time'] = otp_suggestions['otp_end_time'] - pd.Timedelta('10800 s')
otp_suggestions['route'] = otp_suggestions['route'].astype(str)
otp_suggestions['route'] = np.where(otp_suggestions['mode'] == 'BUS',
                                    otp_suggestions['route'].astype(str).str.replace("\.0",'').str.zfill(3),
                                    otp_suggestions['route'])

In [11]:
otp_suggestions['date'][0] == pd.to_datetime(trips_origins['o_boarding_datetime'].dt.strftime('%Y-%m-%d')[0])

True

In [12]:
len(otp_suggestions.drop_duplicates(subset=['user_trip_id']))

956

In [13]:
otp_suggestions.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins
0,2017-05-09,1496,1,1,2017-05-09 05:14:11,2017-05-09 05:17:49,WALK,nan,NaN,NaN,3.633333
1,2017-05-09,1496,1,2,2017-05-09 05:17:50,2017-05-09 05:45:00,BUS,272,27222.0,26365.0,27.166667
2,2017-05-09,1496,1,3,2017-05-09 05:45:00,2017-05-09 05:47:44,WALK,nan,NaN,NaN,2.733333
3,2017-05-09,1496,1,4,2017-05-09 05:55:00,2017-05-09 06:15:56,BUS,304,25891.0,26286.0,20.933333
4,2017-05-09,1496,1,5,2017-05-09 06:15:56,2017-05-09 06:16:35,WALK,nan,NaN,NaN,0.650000


In [14]:
len(otp_suggestions)

43688

#### Adding Parent Stop data to OTP Suggestions

In [15]:
base_gtfs_folder = '/local/tarciso/data/gtfs/'
router_id = get_router_id(otp_suggestions['date'][0])
stops_df = pd.read_csv(base_gtfs_folder + os.sep + router_id + os.sep + 'stops.txt')

In [16]:
stops_df.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,70,104505,Terminal Campina do Siqueira - 303 - Centenári...,Terminal Campina do Siqueira - Campo Comprido,-25.435724,-49.306998,NaN,NaN,0,14506.0,NaN,NaN
1,270,104905,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501341,-49.237597,NaN,NaN,0,14485.0,NaN,NaN
2,276,105606,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451550,-49.214917,NaN,NaN,0,14481.0,NaN,NaN
3,299,105603,Terminal Oficinas - 030 - Interbairros III,Terminal Oficinas - 030 - Interbairros III (Se...,-25.451665,-49.215086,NaN,NaN,0,14481.0,NaN,NaN
4,308,104907,Terminal Carmo - 030 - Interbairros III,Terminal Carmo - 030 - Interbairros III (Senti...,-25.501311,-49.237825,NaN,NaN,0,14485.0,NaN,NaN


In [17]:
stops_parent_stations = stops_df[['stop_id','parent_station']]

In [18]:
otp_suggestions = otp_suggestions.merge(stops_parent_stations, left_on='from_stop_id', right_on='stop_id', how='left') \
                                    .drop(['stop_id'], axis=1)

In [19]:
otp_suggestions.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,2017-05-09,1496,1,1,2017-05-09 05:14:11,2017-05-09 05:17:49,WALK,nan,NaN,NaN,3.633333,NaN
1,2017-05-09,1496,1,2,2017-05-09 05:17:50,2017-05-09 05:45:00,BUS,272,27222,26365.0,27.166667,NaN
2,2017-05-09,1496,1,3,2017-05-09 05:45:00,2017-05-09 05:47:44,WALK,nan,NaN,NaN,2.733333,NaN
3,2017-05-09,1496,1,4,2017-05-09 05:55:00,2017-05-09 06:15:56,BUS,304,25891,26286.0,20.933333,41860.0
4,2017-05-09,1496,1,5,2017-05-09 06:15:56,2017-05-09 06:16:35,WALK,nan,NaN,NaN,0.650000,NaN


#### Selecting trips for whom OTP suggestions were found

In [20]:
selected_trips = trips_origins[trips_origins['o_boarding_id'].isin(otp_suggestions['user_trip_id'])]

In [21]:
len(selected_trips)

920

In [22]:
selected_trips.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,0 days 10:04:22.000000000,6.653455
1,3766348.0,247072,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,...,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,0 days 10:04:22.000000000,6.653455
2,3108318.0,131140,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,...,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,0 days 10:52:10.000000000,10.974491
3,3108318.0,131141,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,...,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,0 days 10:52:10.000000000,10.974491
4,3609013.0,201240,2017-05-09 04:44:28,548,GA205,1.0,31597,2017-05-09 04:44:09,-25.560652,-49.256788,...,2017-05-09 15:07:11,021,08046,NaN,41796,NaT,-25.430289,-49.292615,0 days 10:22:43.000000000,14.935055


In [23]:
itineraries_start = otp_suggestions.query('mode == \'BUS\'') \
                    .groupby(['user_trip_id','itinerary_id']) \
                    .first() \
                    .reset_index()

In [24]:
itineraries_start.head()

,user_trip_id,itinerary_id,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,1496,1,2017-05-09,2,2017-05-09 05:17:50,2017-05-09 05:45:00,BUS,272,27222.0,26365.0,27.166667,41860.0
1,1496,2,2017-05-09,1,2017-05-09 05:03:32,2017-05-09 05:20:00,BUS,226,34486.0,30452.0,16.466667,14471.0
2,1496,3,2017-05-09,2,2017-05-09 05:23:03,2017-05-09 05:55:00,BUS,270,31703.0,26358.0,31.950000,41860.0
3,1496,4,2017-05-09,1,2017-05-09 05:28:22,2017-05-09 05:44:00,BUS,226,34486.0,30452.0,15.633333,14471.0
4,1496,5,2017-05-09,2,2017-05-09 05:40:25,2017-05-09 06:11:00,BUS,271,31703.0,26359.0,30.583333,41860.0


In [25]:
vehicle_boarding_origins = selected_trips[np.logical_not(selected_trips['o_busCode'].str.isdigit())]

In [26]:
len(vehicle_boarding_origins)

555

In [27]:
vehicle_boarding_origins.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,0 days 10:04:22.000000000,6.653455
1,3766348.0,247072,2017-05-09 14:46:17,335,EA302,12.0,33577,2017-05-09 14:45:27,-25.511040,-49.224010,...,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,0 days 10:04:22.000000000,6.653455
2,3108318.0,131140,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,...,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,0 days 10:52:10.000000000,10.974491
4,3609013.0,201240,2017-05-09 04:44:28,548,GA205,1.0,31597,2017-05-09 04:44:09,-25.560652,-49.256788,...,2017-05-09 15:07:11,021,08046,NaN,41796,NaT,-25.430289,-49.292615,0 days 10:22:43.000000000,14.935055
6,1940885.0,42210,2017-05-09 04:44:32,646,HA023,7.0,38521,2017-05-09 12:59:44,-25.565768,-49.314859,...,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,0 days 11:27:07.000000000,6.249052


In [28]:
vehicle_boarding_origins.dtypes

cardNum                            float64
o_boarding_id                        int64
o_boarding_datetime         datetime64[ns]
o_route                             object
o_busCode                           object
o_tripNum                          float64
o_stopPointId                        int64
o_gps_datetime              datetime64[ns]
o_stop_lat                         float64
o_stop_lon                         float64
next_o_boarding_id                   int64
next_o_boarding_datetime    datetime64[ns]
next_o_route                        object
next_o_busCode                      object
next_o_tripNum                     float64
next_o_stopPointId                   int64
next_o_gps_datetime         datetime64[ns]
next_o_stop_lat                    float64
next_o_stop_lon                    float64
boardings_timediff                  object
dist_between_origins               float64
dtype: object

In [29]:
itineraries_start.dtypes

user_trip_id                  int64
itinerary_id                  int64
date                 datetime64[ns]
leg_id                        int64
otp_start_time       datetime64[ns]
otp_end_time         datetime64[ns]
mode                         object
route                        object
from_stop_id                float64
to_stop_id                  float64
otp_duration_mins           float64
parent_station              float64
dtype: object

In [30]:
matched_vehicle_boardings = vehicle_boarding_origins.merge(itineraries_start, left_on=['o_boarding_id','o_route','o_stopPointId'], right_on=['user_trip_id','route','from_stop_id'], how='inner')

In [31]:
num_matched_vehicle_boardings = len(matched_vehicle_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))

print "Vehicle boardings with matching OTP suggestions: ", num_matched_vehicle_boardings, "(", \
                                                            100*(num_matched_vehicle_boardings/float(len(vehicle_boarding_origins))), "%)"

Vehicle boardings with matching OTP suggestions:  498 ( 89.7297297297 %)


In [32]:
matched_vehicle_boardings.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593.0,31572.0,12.350000,14489.0
1,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593.0,31572.0,11.416667,14489.0
2,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593.0,4661.0,22.466667,14492.0
3,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593.0,4661.0,22.466667,14492.0
4,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:31:32,2017-05-09 05:42:57,BUS,548,31593.0,31572.0,11.416667,NaN


In [33]:
pd.concat([matched_vehicle_boardings[vehicle_boarding_origins.columns],vehicle_boarding_origins]).drop_duplicates(keep=False)

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
6,1940885.0,42210,2017-05-09 04:44:32,646,HA023,7.0,38521,2017-05-09 12:59:44,-25.565768,-49.314859,...,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,0 days 11:27:07.000000000,6.249052
20,2178773.0,54396,2017-05-09 04:46:13,619,HA600,5.0,30434,2017-05-09 12:57:05,-25.511661,-49.324223,...,2017-05-09 05:57:54,000,03044,NaN,41826,NaT,-25.439459,-49.271565,0 days 01:11:41.000000000,9.612407
42,3491154.0,178753,2017-05-09 05:54:25,967,MN600,1.0,33734,2017-05-09 05:49:32,-25.426369,-49.277005,...,2017-05-09 15:10:03,967,MN604,10.0,33397,2017-05-09 15:08:47,-25.408069,-49.304026,0 days 09:15:38.000000000,3.391939
92,3510903.0,182201,2017-05-09 05:17:29,040,HB606,1.0,35108,2017-05-09 05:16:53,-25.512566,-49.330333,...,2017-05-09 04:48:58,619,HA600,1.0,36284,2017-05-09 04:48:47,-25.573642,-49.332666,0 days 00:28:31.000000000,6.795384
141,2258052.0,60146,2017-05-09 18:44:40,870,BC189,10.0,29420,2017-05-09 18:39:00,-25.430894,-49.276454,...,2017-05-09 04:51:41,040,MB604,1.0,34260,2017-05-09 04:50:48,-25.423548,-49.351123,0 days 13:52:59.000000000,7.542864
143,3620341.0,204008,2017-05-09 05:54:30,967,MN600,1.0,33734,2017-05-09 05:49:32,-25.426369,-49.277005,...,2017-05-09 14:20:26,901,MC076,9.0,33676,2017-05-09 14:20:13,-25.415973,-49.296807,0 days 08:25:56.000000000,2.300296
204,2113177.0,50896,2017-05-09 04:53:48,643,GA003,7.0,38025,2017-05-09 12:59:43,-25.585504,-49.274348,...,2017-05-09 05:16:13,000,09048,NaN,41829,NaT,-25.527912,-49.291347,0 days 00:22:25.000000000,6.627091
242,3207482.0,140017,2017-05-09 04:55:08,542,GA124,1.0,31262,2017-05-09 04:54:19,-25.544890,-49.265160,...,2017-05-09 16:08:12,000,03041,NaN,26095,NaT,-25.443355,-49.281734,0 days 11:13:04.000000000,11.412121
249,3742409.0,238100,2017-05-09 14:50:44,650,HA030,9.0,39826,2017-05-09 14:49:47,-25.549416,-49.324263,...,2017-05-09 15:42:33,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,0 days 00:51:49.000000000,5.045196
286,1342841.0,18334,2017-05-09 04:56:09,542,GA124,1.0,31263,2017-05-09 04:56:00,-25.541510,-49.270560,...,2017-05-09 16:04:44,021,08044,NaN,41791,NaT,-25.431951,-49.296491,0 days 11:08:35.000000000,12.457360


In [34]:
itineraries_start[itineraries_start['user_trip_id'] == 60146]

,user_trip_id,itinerary_id,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
2916,60146,1,2017-05-09,2,2017-05-09 19:04:27,2017-05-09 19:33:29,BUS,870,28604.0,34258.0,29.033333,NaN
2917,60146,2,2017-05-09,2,2017-05-09 18:55:49,2017-05-09 19:27:52,BUS,876,28604.0,30333.0,32.050000,NaN
2918,60146,3,2017-05-09,2,2017-05-09 18:56:09,2017-05-09 19:10:16,BUS,303,25557.0,70.0,14.116667,41768.0
2919,60146,4,2017-05-09,2,2017-05-09 19:21:27,2017-05-09 19:50:29,BUS,870,28604.0,34258.0,29.033333,NaN
2920,60146,5,2017-05-09,2,2017-05-09 18:53:41,2017-05-09 19:35:29,BUS,972,26720.0,33342.0,41.800000,NaN
2921,60146,6,2017-05-09,2,2017-05-09 19:38:27,2017-05-09 20:07:29,BUS,870,28604.0,34258.0,29.033333,NaN
2922,60146,7,2017-05-09,2,2017-05-09 19:55:34,2017-05-09 20:27:01,BUS,870,28604.0,34258.0,31.450000,NaN
2923,60146,8,2017-05-09,2,2017-05-09 19:56:31,2017-05-09 20:23:13,BUS,876,28604.0,30333.0,26.700000,NaN
2924,60146,9,2017-05-09,2,2017-05-09 20:31:12,2017-05-09 20:55:24,BUS,870,28604.0,34258.0,24.200000,NaN
2925,60146,10,2017-05-09,2,2017-05-09 20:51:12,2017-05-09 21:15:24,BUS,870,28604.0,34258.0,24.200000,NaN


In [35]:
terminal_boarding_origins = selected_trips[selected_trips['o_busCode'].str.isdigit()]

In [36]:
terminal_boarding_origins['o_route'].value_counts()

000    303
021     37
TPH      9
TMA      4
TCJ      2
TFA      2
TCR      2
TSP      2
TCA      1
TSQ      1
TSB      1
TSF      1
Name: o_route, dtype: int64

In [37]:
terminal_021_origins = selected_trips[(selected_trips['o_busCode'].str.isdigit()) & (selected_trips['o_route'] == '021')]

In [38]:
terminal_021_origins.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
5,3609013.0,201241,2017-05-09 15:07:11,021,08046,NaN,41796,NaT,-25.430289,-49.292615,...,2017-05-09 04:44:28,548,GA205,1.0,31597,2017-05-09 04:44:09,-25.560652,-49.256788,0 days 10:22:43.000000000,14.935055
66,1598789.0,27398,2017-05-09 16:39:51,021,08042,NaN,41765,NaT,-25.433218,-49.299969,...,2017-05-09 04:47:57,655,HA257,9.0,39262,2017-05-09 12:59:51,-25.568612,-49.319995,0 days 11:51:54.000000000,15.188683
72,1886552.0,39744,2017-05-09 16:00:02,021,04000,NaN,14506,NaT,-25.436086,-49.307596,...,2017-05-09 04:48:22,650,HA243,1.0,36293,2017-05-09 04:47:57,-25.568989,-49.332253,0 days 11:11:40.000000000,14.983937
90,2385286.0,76681,2017-05-09 15:07:35,021,08046,NaN,41796,NaT,-25.430289,-49.292615,...,2017-05-09 04:48:56,652,LA007,1.0,36697,2017-05-09 04:48:53,-25.532192,-49.337116,0 days 10:18:39.000000000,12.179731
149,1163918.0,12638,2017-05-09 14:32:14,021,08033,NaN,41760,NaT,-25.437598,-49.324377,...,2017-05-09 04:51:59,684,HA602,1.0,39377,2017-05-09 04:51:51,-25.586030,-49.334810,0 days 09:40:15.000000000,16.538020


In [39]:
matched_021_terminal_boardings = terminal_021_origins.merge(itineraries_start, left_on=['o_boarding_id','o_route','o_stopPointId'], right_on=['user_trip_id','route','parent_station'], how='inner')

In [40]:
num_matched_021_terminal_boardings = len(matched_021_terminal_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))

print "Line 021 Terminal boardings with matching OTP suggestions: ", num_matched_021_terminal_boardings, "(", \
                                                            100*(num_matched_021_terminal_boardings/float(len(terminal_021_origins))), "%)"

Line 021 Terminal boardings with matching OTP suggestions:  1 ( 2.7027027027 %)


In [41]:
matched_021_terminal_boardings.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,1886552.0,39744,2017-05-09 16:00:02,021,04000,NaN,14506,NaT,-25.436086,-49.307596,...,2017-05-09,2,2017-05-09 16:04:00,2017-05-09 16:36:00,BUS,021,28136.0,27475.0,32.00,14506.0
1,1886552.0,39744,2017-05-09 16:00:02,021,04000,NaN,14506,NaT,-25.436086,-49.307596,...,2017-05-09,2,2017-05-09 16:18:00,2017-05-09 16:45:21,BUS,021,28136.0,33044.0,27.35,14506.0
2,1886552.0,39744,2017-05-09 16:00:02,021,04000,NaN,14506,NaT,-25.436086,-49.307596,...,2017-05-09,2,2017-05-09 16:27:00,2017-05-09 16:57:00,BUS,021,28136.0,27475.0,30.00,14506.0


In [42]:
terminal_boarding_origins = selected_trips[(selected_trips['o_busCode'].str.isdigit()) & (selected_trips['o_route'] != '021')]

In [43]:
terminal_boarding_origins.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,next_o_boarding_datetime,next_o_route,next_o_busCode,next_o_tripNum,next_o_stopPointId,next_o_gps_datetime,next_o_stop_lat,next_o_stop_lon,boardings_timediff,dist_between_origins
3,3108318.0,131141,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,...,2017-05-09 04:43:59,548,GA205,1.0,44395,2017-05-09 04:43:29,-25.558834,-49.258696,0 days 10:52:10.000000000,10.974491
7,1940885.0,42211,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,...,2017-05-09 04:44:32,646,HA023,7.0,38521,2017-05-09 12:59:44,-25.565768,-49.314859,0 days 11:27:07.000000000,6.249052
9,1232958.0,14668,2017-05-09 15:04:54,000,00024,NaN,43347,NaT,-25.475898,-49.249983,...,2017-05-09 04:45:04,548,GA205,1.0,48762,2017-05-09 04:44:52,-25.563193,-49.253670,0 days 10:19:50.000000000,9.713823
11,715315.0,2902,2017-05-09 13:33:44,000,09070,NaN,43723,NaT,-25.433715,-49.270258,...,2017-05-09 04:45:14,684,HA602,1.0,39368,2017-05-09 04:43:41,-25.588450,-49.341190,0 days 08:48:30.000000000,18.620097
13,1982410.0,44255,2017-05-09 18:38:20,000,00070,NaN,14487,NaT,-25.481927,-49.246999,...,2017-05-09 04:45:28,684,HA602,1.0,39369,2017-05-09 04:45:27,-25.592110,-49.342120,0 days 13:52:52.000000000,15.530264


In [44]:
matched_terminal_boardings = terminal_boarding_origins.merge(itineraries_start, left_on=['o_boarding_id','o_stopPointId'], right_on=['user_trip_id','parent_station'], how='inner') \
                .drop_duplicates(subset=['cardNum','o_boarding_id'])

In [45]:
num_matched_terminal_boardings = len(matched_terminal_boardings.drop_duplicates(subset=['cardNum','o_boarding_id']))

print "Terminal boardings with matching OTP suggestions: ", num_matched_terminal_boardings, "(", \
                                                            100*(num_matched_terminal_boardings/float(len(terminal_boarding_origins))), "%)"

Terminal boardings with matching OTP suggestions:  234 ( 71.3414634146 %)


In [46]:
matched_terminal_boardings.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,3108318.0,131141,2017-05-09 15:36:09,000,09029,NaN,26121,NaT,-25.464630,-49.291315,...,2017-05-09,2,2017-05-09 15:41:31,2017-05-09 16:06:00,BUS,603,25398.0,26296.0,24.483333,26121.0
9,1940885.0,42211,2017-05-09 16:11:39,TPH,03019,NaN,14494,NaT,-25.512487,-49.295048,...,2017-05-09,1,2017-05-09 16:13:00,2017-05-09 16:33:37,BUS,617,31051.0,38521.0,20.616667,14494.0
19,1232958.0,14668,2017-05-09 15:04:54,000,00024,NaN,43347,NaT,-25.475898,-49.249983,...,2017-05-09,2,2017-05-09 15:05:39,2017-05-09 15:08:41,BUS,602,25468.0,25463.0,3.033333,43347.0
23,1982410.0,44255,2017-05-09 18:38:20,000,00070,NaN,14487,NaT,-25.481927,-49.246999,...,2017-05-09,2,2017-05-09 18:40:00,2017-05-09 18:45:41,BUS,602,25474.0,25463.0,5.683333,14487.0
33,2178773.0,54398,2017-05-09 15:41:51,000,06057,NaN,26011,NaT,-25.435634,-49.256611,...,2017-05-09,2,2017-05-09 15:48:26,2017-05-09 15:55:00,BUS,303,25646.0,26185.0,6.566667,26011.0


In [47]:
total_num_matches = num_matched_vehicle_boardings + num_matched_021_terminal_boardings + num_matched_terminal_boardings

print "Total number of matches: ", total_num_matches, "(", \
                                    100*(total_num_matches/float(len(selected_trips))), "%)"

Total number of matches:  733 ( 79.6739130435 %)


In [48]:
boarding_suggestions_matches = pd.concat([matched_vehicle_boardings,matched_021_terminal_boardings,matched_terminal_boardings])

In [49]:
len(boarding_suggestions_matches)

2306

In [50]:
boarding_suggestions_matches.head()

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,date,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593.0,31572.0,12.350000,14489.0
1,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593.0,31572.0,11.416667,14489.0
2,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593.0,4661.0,22.466667,14492.0
3,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593.0,4661.0,22.466667,14492.0
4,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.55871,-49.26409,...,2017-05-09,1,2017-05-09 05:31:32,2017-05-09 05:42:57,BUS,548,31593.0,31572.0,11.416667,NaN


#### Add OTP extra origin/next-origin pairs to final dataset

In [51]:
boarding_suggestions_matches.dtypes

cardNum                            float64
o_boarding_id                        int64
o_boarding_datetime         datetime64[ns]
o_route                             object
o_busCode                           object
o_tripNum                          float64
o_stopPointId                       object
o_gps_datetime              datetime64[ns]
o_stop_lat                         float64
o_stop_lon                         float64
next_o_boarding_id                   int64
next_o_boarding_datetime    datetime64[ns]
next_o_route                        object
next_o_busCode                      object
next_o_tripNum                     float64
next_o_stopPointId                   int64
next_o_gps_datetime         datetime64[ns]
next_o_stop_lat                    float64
next_o_stop_lon                    float64
boardings_timediff                  object
dist_between_origins               float64
user_trip_id                         int64
itinerary_id                         int64
date       

In [54]:
otp_legs_suggestions_matches = boarding_suggestions_matches[np.append(trips_origins.columns.values,['itinerary_id'])] \
                    .merge(otp_suggestions, left_on=['o_boarding_id','itinerary_id'], right_on=['user_trip_id','itinerary_id'], how='inner') \
                    .query('mode == \'BUS\'')

In [55]:
len(otp_legs_suggestions_matches)

5250

In [56]:
otp_legs_suggestions_matches

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,user_trip_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593,31572.0,12.350000,NaN
2,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,3,2017-05-09 05:04:44,2017-05-09 05:22:00,BUS,535,31571,30030.0,17.266667,NaN
4,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,5,2017-05-09 05:30:00,2017-05-09 05:33:55,BUS,335,30027,33576.0,3.916667,14489.0
6,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593,31572.0,11.416667,NaN
8,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,3,2017-05-09 05:19:19,2017-05-09 05:35:00,BUS,535,31571,30030.0,15.683333,NaN
10,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,5,2017-05-09 05:45:00,2017-05-09 05:48:08,BUS,335,30027,33576.0,3.133333,14489.0
12,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593,4661.0,22.466667,NaN
14,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,3,2017-05-09 05:30:00,2017-05-09 05:45:00,BUS,602,27121,27352.0,15.000000,14492.0
16,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593,4661.0,22.466667,NaN
18,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,247071,3,2017-05-09 05:48:00,2017-05-09 06:01:00,BUS,508,26257,30025.0,13.000000,14492.0


In [57]:
otp_legs_suggestions_matches['first_vehicle_boarding'] = np.where(np.logical_not(otp_legs_suggestions_matches['o_busCode'].str.isdigit()) & 
                                                                      (otp_legs_suggestions_matches['o_route'] == otp_legs_suggestions_matches['route']),
                                                                      True,
                                                                      False)

In [58]:
otp_legs_suggestions_matches

,cardNum,o_boarding_id,o_boarding_datetime,o_route,o_busCode,o_tripNum,o_stopPointId,o_gps_datetime,o_stop_lat,o_stop_lon,...,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station,first_vehicle_boarding
0,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593,31572.0,12.350000,NaN,True
2,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,3,2017-05-09 05:04:44,2017-05-09 05:22:00,BUS,535,31571,30030.0,17.266667,NaN,False
4,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,5,2017-05-09 05:30:00,2017-05-09 05:33:55,BUS,335,30027,33576.0,3.916667,14489.0,False
6,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593,31572.0,11.416667,NaN,True
8,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,3,2017-05-09 05:19:19,2017-05-09 05:35:00,BUS,535,31571,30030.0,15.683333,NaN,False
10,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,5,2017-05-09 05:45:00,2017-05-09 05:48:08,BUS,335,30027,33576.0,3.133333,14489.0,False
12,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593,4661.0,22.466667,NaN,True
14,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,3,2017-05-09 05:30:00,2017-05-09 05:45:00,BUS,602,27121,27352.0,15.000000,14492.0,False
16,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593,4661.0,22.466667,NaN,True
18,3766348.0,247071,2017-05-09 04:41:55,548,GA205,1.0,31593,2017-05-09 04:41:19,-25.558710,-49.264090,...,3,2017-05-09 05:48:00,2017-05-09 06:01:00,BUS,508,26257,30025.0,13.000000,14492.0,False


In [59]:
otp_filtered_legs = otp_legs_suggestions_matches.filter(items=np.append(otp_suggestions.columns.values,['first_vehicle_boarding','o_busCode','o_tripNum','o_boarding_datetime']))

In [60]:
otp_filtered_legs

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station,first_vehicle_boarding,o_busCode,o_tripNum,o_boarding_datetime
0,2017-05-09,247071,1,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593,31572.0,12.350000,NaN,True,GA205,1.0,2017-05-09 04:41:55
2,2017-05-09,247071,1,3,2017-05-09 05:04:44,2017-05-09 05:22:00,BUS,535,31571,30030.0,17.266667,NaN,False,GA205,1.0,2017-05-09 04:41:55
4,2017-05-09,247071,1,5,2017-05-09 05:30:00,2017-05-09 05:33:55,BUS,335,30027,33576.0,3.916667,14489.0,False,GA205,1.0,2017-05-09 04:41:55
6,2017-05-09,247071,2,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593,31572.0,11.416667,NaN,True,GA205,1.0,2017-05-09 04:41:55
8,2017-05-09,247071,2,3,2017-05-09 05:19:19,2017-05-09 05:35:00,BUS,535,31571,30030.0,15.683333,NaN,False,GA205,1.0,2017-05-09 04:41:55
10,2017-05-09,247071,2,5,2017-05-09 05:45:00,2017-05-09 05:48:08,BUS,335,30027,33576.0,3.133333,14489.0,False,GA205,1.0,2017-05-09 04:41:55
12,2017-05-09,247071,3,1,2017-05-09 04:55:32,2017-05-09 05:18:00,BUS,548,31593,4661.0,22.466667,NaN,True,GA205,1.0,2017-05-09 04:41:55
14,2017-05-09,247071,3,3,2017-05-09 05:30:00,2017-05-09 05:45:00,BUS,602,27121,27352.0,15.000000,14492.0,False,GA205,1.0,2017-05-09 04:41:55
16,2017-05-09,247071,5,1,2017-05-09 05:19:32,2017-05-09 05:42:00,BUS,548,31593,4661.0,22.466667,NaN,True,GA205,1.0,2017-05-09 04:41:55
18,2017-05-09,247071,5,3,2017-05-09 05:48:00,2017-05-09 06:01:00,BUS,508,26257,30025.0,13.000000,14492.0,False,GA205,1.0,2017-05-09 04:41:55


#### Find OTP Suggested Itineraries in BUSTE Data

In [61]:
bus_trips = pd.read_csv('/local/tarciso/masters/data/bus_trips/test/enhanced-buste/2017_05_09_bus_trips.csv', parse_dates=['gps_datetime'])

/local/tarciso/programs/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
bus_trips = bus_trips.sort_values(['route','busCode','tripNum','gps_datetime'])
bus_trips['route'] = bus_trips['route'].astype(str).str.replace("\.0",'').str.zfill(3)

In [63]:
bus_trips.head()

,route,busCode,tripNum,stopPointId,gps_datetime,stop_lat,stop_lon,parent_station
140540,001,BN997,1.0,29887,2017-05-09 06:47:54,-25.428171,-49.264558,NaN
140541,001,BN997,1.0,31453,2017-05-09 06:48:54,-25.430388,-49.263602,NaN
138976,001,BN997,1.0,31454,2017-05-09 06:49:52,-25.433503,-49.262257,NaN
140542,001,BN997,1.0,30748,2017-05-09 06:52:21,-25.435187,-49.264933,NaN
140543,001,BN997,1.0,30749,2017-05-09 06:52:35,-25.435959,-49.266889,NaN


In [64]:
otp_filtered_legs.head()

,date,user_trip_id,itinerary_id,leg_id,otp_start_time,otp_end_time,mode,route,from_stop_id,to_stop_id,otp_duration_mins,parent_station,first_vehicle_boarding,o_busCode,o_tripNum,o_boarding_datetime
0,2017-05-09,247071,1,1,2017-05-09 04:41:40,2017-05-09 04:54:01,BUS,548,31593,31572.0,12.350000,NaN,True,GA205,1.0,2017-05-09 04:41:55
2,2017-05-09,247071,1,3,2017-05-09 05:04:44,2017-05-09 05:22:00,BUS,535,31571,30030.0,17.266667,NaN,False,GA205,1.0,2017-05-09 04:41:55
4,2017-05-09,247071,1,5,2017-05-09 05:30:00,2017-05-09 05:33:55,BUS,335,30027,33576.0,3.916667,14489.0,False,GA205,1.0,2017-05-09 04:41:55
6,2017-05-09,247071,2,1,2017-05-09 05:07:32,2017-05-09 05:18:57,BUS,548,31593,31572.0,11.416667,NaN,True,GA205,1.0,2017-05-09 04:41:55
8,2017-05-09,247071,2,3,2017-05-09 05:19:19,2017-05-09 05:35:00,BUS,535,31571,30030.0,15.683333,NaN,False,GA205,1.0,2017-05-09 04:41:55


#### Find Legs BUSTE match using current and previous leg information

In [91]:
otp_buste_df = pd.DataFrame()

for i in range(len(otp_filtered_legs.head(5))):
    curr_leg = otp_filtered_legs.iloc[i:i+1]
    prev_leg_end_time = np.nan
    if ((otp_filtered_legs.user_trip_id.iloc[i-1] == otp_filtered_legs.user_trip_id.iloc[i]) and 
        (otp_filtered_legs.itinerary_id.iloc[i-1] == otp_filtered_legs.itinerary_id.iloc[i])):
        prev_leg_end_time = otp_filtered_legs.otp_end_time.iloc[i-1]
    else:
        prev_leg_end_time = otp_filtered_legs.otp_end_time.iloc[0].floor('h')
    
    #print curr_leg, prev_leg_end_time
    
    #Match OTP Start with BUSTE
    otp_buste_leg = curr_leg.merge(bus_trips, 
                         left_on=['route','from_stop_id'], 
                         right_on=['route','stopPointId'], 
                         how='inner') \
                    .where(lambda x: x['gps_datetime'] > prev_leg_end_time) \
                    .assign(otp_buste_start_timediff = lambda x: np.absolute(x['gps_datetime'] - x['otp_start_time'])) \
                    .sort_values(['user_trip_id','itinerary_id','leg_id','otp_buste_start_timediff']) \
                    .groupby(['user_trip_id','itinerary_id','leg_id']) \
                    .first() \
                    .reset_index() \
                    .filter(['user_trip_id','first_vehicle_boarding','itinerary_id','leg_id','route',
                             'busCode','o_busCode','tripNum','o_tripNum','from_stop_id','otp_start_time',
                             'gps_datetime','o_boarding_datetime','otp_buste_start_timediff','to_stop_id',
                             'otp_end_time']) \
                    .rename(index=str, columns={'to_stop_id':'stopPointId', 'gps_datetime':'matched_start_time'}) \
                    .merge(bus_trips, 
                             on=['route','busCode','tripNum','stopPointId'], 
                             how='inner') \
                    .assign(otp_buste_end_timediff = lambda x: np.absolute(x['gps_datetime'] - x['otp_end_time'])) \
                    .sort_values(['user_trip_id','itinerary_id','leg_id','otp_buste_end_timediff']) \
                    .groupby(['user_trip_id','itinerary_id','leg_id']) \
                    .first() \
                    .reset_index() \
                    .rename(index=str, columns={'stopPointId':'to_stop_id', 'gps_datetime':'matched_end_time'}) \
                    .assign(leg_duration = lambda x: x['matched_end_time'] - x['matched_start_time'],
                            boarding_otp_match_start_timediff = lambda x: np.absolute(x['o_boarding_datetime'] - x['matched_start_time'])) \
                    .filter(['user_trip_id','first_vehicle_boarding','itinerary_id','leg_id','route','busCode',
                             'o_busCode','tripNum','o_tripNum','from_stop_id','otp_start_time',
                             'matched_start_time','o_boarding_datetime','otp_buste_start_timediff','to_stop_id',
                             'otp_end_time','matched_end_time','otp_buste_end_timediff', 
                             'boarding_otp_match_start_timediff', 'leg_duration'])
    
    otp_buste_df = pd.concat([otp_buste_df,otp_buste_leg])
    
    
otp_buste_df = otp_buste_df.sort_values(['user_trip_id','itinerary_id','leg_id']) \
                            .filter(['user_trip_id','itinerary_id','leg_id','otp_start_time','matched_start_time','otp_end_time','matched_end_time'])
otp_buste_df
    
    

,user_trip_id,itinerary_id,leg_id,otp_start_time,matched_start_time,otp_end_time,matched_end_time
0,247071.0,1.0,1.0,2017-05-09 04:41:40,2017-05-09 04:41:19,2017-05-09 04:54:01,2017-05-09 04:52:14
0,247071.0,1.0,5.0,2017-05-09 05:30:00,2017-05-09 05:23:39,2017-05-09 05:33:55,2017-05-09 05:32:54
0,247071.0,2.0,1.0,2017-05-09 05:07:32,2017-05-09 04:55:41,2017-05-09 05:18:57,2017-05-09 05:06:01
0,247071.0,2.0,3.0,2017-05-09 05:19:19,2017-05-09 05:19:29,2017-05-09 05:35:00,2017-05-09 05:35:12


#### Old Approach

In [ ]:
otp_legs_buste_start = otp_filtered_legs.merge(bus_trips, 
                                 left_on=['route','from_stop_id'], 
                                 right_on=['route','stopPointId'], 
                                 how='inner') \
                        .assign(otp_buste_start_timediff = lambda x: np.absolute(x['gps_datetime'] - x['otp_start_time'])) \
                        .sort_values(['user_trip_id','itinerary_id','leg_id','otp_buste_start_timediff']) \
                        .filter(['user_trip_id','first_vehicle_boarding','itinerary_id','leg_id','route','busCode','o_busCode','tripNum','o_tripNum','from_stop_id','otp_start_time','gps_datetime','o_boarding_datetime','otp_buste_start_timediff','to_stop_id','otp_end_time'])
            
otp_legs_buste_start = otp_legs_buste_start[otp_legs_buste_start['otp_buste_start_timediff'] < pd.Timedelta('20min')]

In [80]:
len(otp_legs_buste_start)

27156

In [81]:
otp_legs_buste_start.head()

,user_trip_id,first_vehicle_boarding,itinerary_id,leg_id,route,busCode,o_busCode,tripNum,o_tripNum,from_stop_id,otp_start_time,gps_datetime,o_boarding_datetime,otp_buste_start_timediff,to_stop_id,otp_end_time
492047,1496,True,2,1,226,BA140,BA140,1.0,1.0,34486,2017-05-09 05:03:32,2017-05-09 05:03:32,2017-05-09 05:04:34,00:00:00,30452.0,2017-05-09 05:20:00
492048,1496,True,2,1,226,BA140,BA140,1.0,1.0,34486,2017-05-09 05:03:32,2017-05-09 05:03:32,2017-05-09 05:04:34,00:00:00,30452.0,2017-05-09 05:20:00
469449,1496,False,2,3,203,BD131,BA140,1.0,1.0,26190,2017-05-09 05:25:46,2017-05-09 05:26:08,2017-05-09 05:04:34,00:00:22,25389.0,2017-05-09 05:45:54
469520,1496,False,2,3,203,BD148,BA140,1.0,1.0,26190,2017-05-09 05:25:46,2017-05-09 05:33:38,2017-05-09 05:04:34,00:07:52,25389.0,2017-05-09 05:45:54
469618,1496,False,2,3,203,BE848,BA140,1.0,1.0,26190,2017-05-09 05:25:46,2017-05-09 05:15:17,2017-05-09 05:04:34,00:10:29,25389.0,2017-05-09 05:45:54


In [82]:
otp_legs_buste_start.otp_buste_start_timediff.describe()

count                     27156
mean     0 days 00:09:27.285424
std      0 days 00:06:20.570684
min             0 days 00:00:00
25%             0 days 00:02:59
50%             0 days 00:09:38
75%             0 days 00:14:52
max             0 days 00:19:59
Name: otp_buste_start_timediff, dtype: object

In [84]:
otp_legs_buste = otp_legs_buste_start \
                        .rename(index=str, columns={'to_stop_id':'stopPointId', 'gps_datetime':'matched_start_time'}) \
                        .merge(bus_trips, 
                                 on=['route','busCode','tripNum','stopPointId'], 
                                 how='inner') \
                        .assign(otp_buste_end_timediff = lambda x: np.absolute(x['gps_datetime'] - x['otp_end_time'])) \
                        .sort_values(['user_trip_id','itinerary_id','leg_id','otp_buste_end_timediff']) \
                        .rename(index=str, columns={'stopPointId':'to_stop_id', 'gps_datetime':'matched_end_time'}) \
                        .assign(leg_duration = lambda x: x['matched_end_time'] - x['matched_start_time'],
                                boarding_otp_match_start_timediff = lambda x: np.absolute(x['o_boarding_datetime'] - x['matched_start_time'])) \
                        .filter(['user_trip_id','first_vehicle_boarding','itinerary_id','leg_id','route','busCode','o_busCode','tripNum','o_tripNum','from_stop_id','otp_start_time','matched_start_time','o_boarding_datetime','otp_buste_start_timediff','to_stop_id','otp_end_time','matched_end_time','otp_buste_end_timediff', 'boarding_otp_match_start_timediff', 'leg_duration'])

otp_legs_buste = otp_legs_buste[otp_legs_buste['otp_buste_end_timediff'] < pd.Timedelta('20min')]

In [85]:
len(otp_legs_buste)

46033

In [74]:
otp_legs_buste.head(20)

,user_trip_id,first_vehicle_boarding,itinerary_id,leg_id,route,busCode,o_busCode,tripNum,o_tripNum,from_stop_id,otp_start_time,matched_start_time,o_boarding_datetime,otp_buste_start_timediff,to_stop_id,otp_end_time,matched_end_time,otp_buste_end_timediff,boarding_otp_match_start_timediff,leg_duration
0,1496,True,2,1,226,BA140,BA140,1.0,1.0,34486.0,2017-05-09 05:03:32,2017-05-09 05:03:32,2017-05-09 05:04:34,00:00:00,30452.0,2017-05-09 05:20:00,2017-05-09 05:14:02,00:05:58,00:01:02,00:10:30
1,1496,False,2,3,203,BD131,BA140,1.0,1.0,26190.0,2017-05-09 05:25:46,2017-05-09 05:26:08,2017-05-09 05:04:34,00:00:22,25389.0,2017-05-09 05:45:54,2017-05-09 05:47:45,00:01:51,00:21:34,00:21:37
2,1496,False,2,4,303,DE701,BA140,1.0,1.0,25389.0,2017-05-09 05:49:21,2017-05-09 05:44:38,2017-05-09 05:04:34,00:04:43,26195.0,2017-05-09 06:20:00,2017-05-09 06:14:56,00:05:04,00:40:04,00:30:18
3,1496,False,2,6,732,JA024,BA140,1.0,1.0,31002.0,2017-05-09 06:23:00,2017-05-09 06:18:42,2017-05-09 05:04:34,00:04:18,33185.0,2017-05-09 06:30:19,2017-05-09 06:29:58,00:00:21,01:14:08,00:11:16
4,1496,True,4,1,226,BA140,BA140,2.0,1.0,34486.0,2017-05-09 05:28:22,2017-05-09 05:45:57,2017-05-09 05:04:34,00:17:35,30452.0,2017-05-09 05:44:00,2017-05-09 06:03:33,00:19:33,00:41:23,00:17:36
5,1496,False,4,3,203,BD133,BA140,1.0,1.0,26190.0,2017-05-09 05:48:46,2017-05-09 05:48:38,2017-05-09 05:04:34,00:00:08,26187.0,2017-05-09 05:54:00,2017-05-09 05:56:51,00:02:51,00:44:04,00:08:13
6,1496,False,4,5,023,BL496,BA140,1.0,1.0,26237.0,2017-05-09 05:57:00,2017-05-09 06:10:24,2017-05-09 05:04:34,00:13:24,26283.0,2017-05-09 06:18:00,2017-05-09 06:30:58,00:12:58,01:05:50,00:20:34
7,1496,False,4,7,304,DL300,BA140,1.0,1.0,26282.0,2017-05-09 06:21:13,2017-05-09 06:32:42,2017-05-09 05:04:34,00:11:29,2654.0,2017-05-09 06:37:00,2017-05-09 06:48:00,00:11:00,01:28:08,00:15:18
8,1869,True,2,1,653,HA270,HA270,1.0,1.0,36174.0,2017-05-09 05:03:02,2017-05-09 05:02:18,2017-05-09 05:03:17,00:00:44,30435.0,2017-05-09 05:20:00,2017-05-09 05:17:17,00:02:43,00:00:59,00:14:59
9,1869,False,2,3,612,HA251,HA270,1.0,1.0,30437.0,2017-05-09 05:26:00,2017-05-09 05:22:34,2017-05-09 05:03:17,00:03:26,28389.0,2017-05-09 05:48:00,2017-05-09 05:44:43,00:03:17,00:19:17,00:22:09


In [75]:
otp_legs_buste.otp_buste_end_timediff.describe()

count                      5000
mean     0 days 00:13:58.402200
std      0 days 00:31:20.479308
min             0 days 00:00:00
25%             0 days 00:01:39
50%             0 days 00:04:13
75%      0 days 00:11:44.500000
max             0 days 09:57:28
Name: otp_buste_end_timediff, dtype: object

In [76]:
otp_legs_buste.boarding_otp_match_start_timediff.describe()

count                      5000
mean     0 days 00:56:53.248600
std      0 days 00:46:14.568322
min             0 days 00:00:01
25%             0 days 00:26:06
50%      0 days 00:54:20.500000
75%             0 days 01:20:41
max             0 days 18:19:17
Name: boarding_otp_match_start_timediff, dtype: object

In [77]:
otp_legs_buste.leg_duration.describe()

count                      5000
mean     0 days 00:11:44.382800
std      0 days 00:31:32.508600
min           -1 days +14:02:36
25%      0 days 00:07:53.750000
50%             0 days 00:15:53
75%             0 days 00:25:35
max             0 days 09:25:28
Name: leg_duration, dtype: object

#### Choosing itinerary

#### Adding stops location data

In [78]:
stops_locations = stops_df[['stop_id','stop_lat','stop_lon']]

In [79]:
user_trips_ids = otp_legs_suggestions_matches[['cardNum','user_trip_id']].drop_duplicates().sort_values(['cardNum','user_trip_id'])

In [80]:
otp_legs_buste_data = otp_legs_buste.merge(stops_locations, left_on='from_stop_id', right_on='stop_id', how='left') \
        .drop('stop_id', axis=1) \
        .rename(index=str, columns={'stop_lat':'from_stop_lat','stop_lon':'from_stop_lon'}) \
        .merge(stops_locations, left_on='to_stop_id', right_on='stop_id', how='left') \
        .drop('stop_id', axis=1) \
        .rename(index=str, columns={'stop_lat':'to_stop_lat','stop_lon':'to_stop_lon'}) \
        .merge(user_trips_ids, on=['user_trip_id'], how='inner') \
                            [np.append(np.append(['cardNum'],otp_legs_buste.columns.values),['from_stop_lat','from_stop_lon','to_stop_lat','to_stop_lon'])]

In [81]:
otp_legs_buste_data

,cardNum,user_trip_id,first_vehicle_boarding,itinerary_id,leg_id,route,busCode,o_busCode,tripNum,o_tripNum,...,to_stop_id,otp_end_time,matched_end_time,otp_buste_end_timediff,boarding_otp_match_start_timediff,leg_duration,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
0,568690.0,1496,True,2,1,226,BA140,BA140,1.0,1.0,...,30452,2017-05-09 05:20:00,2017-05-09 05:14:02,00:05:58,00:01:02,00:10:30,-25.373245,-49.244344,-25.393259,-49.240989
1,568690.0,1496,False,2,3,203,BD131,BA140,1.0,1.0,...,25389,2017-05-09 05:45:54,2017-05-09 05:47:45,00:01:51,00:21:34,00:21:37,-25.393517,-49.240616,-25.437593,-49.266727
2,568690.0,1496,False,2,4,303,DE701,BA140,1.0,1.0,...,26195,2017-05-09 06:20:00,2017-05-09 06:14:56,00:05:04,00:40:04,00:30:18,-25.437593,-49.266727,-25.441720,-49.346978
3,568690.0,1496,False,2,6,732,JA024,BA140,1.0,1.0,...,33185,2017-05-09 06:30:19,2017-05-09 06:29:58,00:00:21,01:14:08,00:11:16,-25.441531,-49.346711,-25.451980,-49.362003
4,568690.0,1496,True,4,1,226,BA140,BA140,2.0,1.0,...,30452,2017-05-09 05:44:00,2017-05-09 06:03:33,00:19:33,00:41:23,00:17:36,-25.373245,-49.244344,-25.393259,-49.240989
5,568690.0,1496,False,4,3,203,BD133,BA140,1.0,1.0,...,26187,2017-05-09 05:54:00,2017-05-09 05:56:51,00:02:51,00:44:04,00:08:13,-25.393517,-49.240616,-25.406439,-49.252743
6,568690.0,1496,False,4,5,023,BL496,BA140,1.0,1.0,...,26283,2017-05-09 06:18:00,2017-05-09 06:30:58,00:12:58,01:05:50,00:20:34,-25.406516,-49.253004,-25.435856,-49.307661
7,568690.0,1496,False,4,7,304,DL300,BA140,1.0,1.0,...,2654,2017-05-09 06:37:00,2017-05-09 06:48:00,00:11:00,01:28:08,00:15:18,-25.435742,-49.307359,-25.449626,-49.353856
8,618552.0,1869,True,2,1,653,HA270,HA270,1.0,1.0,...,30435,2017-05-09 05:20:00,2017-05-09 05:17:17,00:02:43,00:00:59,00:14:59,-25.500684,-49.351664,-25.511621,-49.324356
9,618552.0,1869,False,2,3,612,HA251,HA270,1.0,1.0,...,28389,2017-05-09 05:48:00,2017-05-09 05:44:43,00:03:17,00:19:17,00:22:09,-25.511524,-49.324729,-25.476144,-49.292507


In [82]:
otp_buste_itineraries = otp_legs_buste_data \
                            .groupby(['cardNum','user_trip_id','itinerary_id']) \
                            .agg({'from_stop_id': lambda x: x.iloc[0], 
                                  'matched_start_time': lambda x: x.iloc[0], 
                                  'from_stop_lat': lambda x: x.iloc[0], 
                                  'from_stop_lon': lambda x: x.iloc[0],
                                  'to_stop_id': lambda x: x.iloc[-1], 
                                  'matched_end_time': lambda x: x.iloc[-1], 
                                  'to_stop_lat': lambda x: x.iloc[-1], 
                                  'to_stop_lon': lambda x: x.iloc[-1],
                                  'leg_id': lambda x: len(x),
                                  'first_vehicle_boarding' : lambda x: x.any()}) \
                            .rename(index=str, columns={'leg_id':'num_transfers','first_vehicle_boarding':'vehicle_boarding'}) \
                            .add_prefix('match_') \
                            .reset_index() \
                            .assign(cardNum = lambda x: x['cardNum'].astype(float),
                                    user_trip_id = lambda x: x['user_trip_id'].astype(int),
                                    itinerary_id = lambda x: x['itinerary_id'].astype(int))

In [83]:
otp_buste_itineraries_summary = otp_buste_itineraries \
                            .merge(otp_legs_suggestions_matches.drop_duplicates(subset=['cardNum','user_trip_id','itinerary_id']), on=['cardNum','user_trip_id','itinerary_id'], how='inner') \
                            [['cardNum', 'user_trip_id', 'itinerary_id',
                                  'match_from_stop_id', 'match_matched_start_time', 'o_boarding_datetime',
                                  'match_from_stop_lat', 'match_from_stop_lon', 'o_stop_lat', 'o_stop_lon',
                                  'match_to_stop_id', 'match_matched_end_time', 'next_o_boarding_datetime',
                                  'match_to_stop_lat', 'match_to_stop_lon', 'next_o_stop_lat', 'next_o_stop_lon','match_num_transfers', 'match_vehicle_boarding']] \
                            .assign(start_diff = lambda x: np.absolute(x['match_matched_start_time'] - x['o_boarding_datetime']),
                                    trip_duration = lambda x: x['match_matched_end_time'] - x['match_matched_start_time'],
                                    origin_dist = lambda x: dist(x['match_from_stop_lat'], x['match_from_stop_lon'], x['o_stop_lat'], x['o_stop_lon']),
                                    next_origin_dist = lambda x: dist(x['match_to_stop_lat'], x['match_to_stop_lon'], x['next_o_stop_lat'], x['next_o_stop_lon'])) \
                            .sort_values(['cardNum','user_trip_id'])

/local/tarciso/programs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in arccos
  """


In [84]:
otp_buste_itineraries_summary.head()

,cardNum,user_trip_id,itinerary_id,match_from_stop_id,match_matched_start_time,o_boarding_datetime,match_from_stop_lat,match_from_stop_lon,o_stop_lat,o_stop_lon,...,match_to_stop_lat,match_to_stop_lon,next_o_stop_lat,next_o_stop_lon,match_num_transfers,match_vehicle_boarding,next_origin_dist,origin_dist,start_diff,trip_duration
0,568690.0,1496,2,34486.0,2017-05-09 05:03:32,2017-05-09 05:04:34,-25.373245,-49.244344,-25.373245,-49.244344,...,-25.451980,-49.362003,-25.451781,-49.361863,4,True,0.026183,0.0,00:01:02,01:26:26
1,568690.0,1496,4,34486.0,2017-05-09 05:45:57,2017-05-09 05:04:34,-25.373245,-49.244344,-25.373245,-49.244344,...,-25.449626,-49.353856,-25.451781,-49.361863,4,True,0.838850,0.0,00:41:23,01:02:03
2,618552.0,1869,2,36174.0,2017-05-09 05:02:18,2017-05-09 05:03:17,-25.500684,-49.351664,-25.500684,-49.351664,...,-25.426190,-49.248740,-25.426190,-49.248740,3,True,0.000000,0.0,00:00:59,01:11:48
3,618552.0,1869,4,36174.0,2017-05-09 05:22:45,2017-05-09 05:03:17,-25.500684,-49.351664,-25.500684,-49.351664,...,-25.426190,-49.248740,-25.426190,-49.248740,3,True,0.000000,0.0,00:19:28,01:06:17
4,618552.0,1869,5,36174.0,2017-05-09 05:22:45,2017-05-09 05:03:17,-25.500684,-49.351664,-25.500684,-49.351664,...,-25.427824,-49.249283,-25.426190,-49.248740,4,True,0.189657,0.0,00:19:28,01:20:24


In [85]:
otp_buste_itineraries_summary.origin_dist.describe(percentiles=[.25,.5,.75,.9,.95,.99])

count    2248.000000
mean        0.141070
std         0.974008
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
90%         0.003530
95%         0.042670
99%         5.076601
max        12.529589
Name: origin_dist, dtype: float64

In [86]:
len(otp_buste_itineraries_summary)

2287

In [87]:
otp_buste_itineraries_summary.match_vehicle_boarding.describe()

count     2287
unique       2
top       True
freq      2007
Name: match_vehicle_boarding, dtype: object

In [88]:
otp_buste_itineraries_summary.drop_duplicates('user_trip_id').match_vehicle_boarding.value_counts()

True     487
False    243
Name: match_vehicle_boarding, dtype: int64

In [89]:
otp_buste_itineraries_filtered = otp_buste_itineraries_summary[((otp_buste_itineraries_summary['trip_duration'] > pd.Timedelta('0s')) & (otp_buste_itineraries_summary['trip_duration'] < pd.Timedelta('2h'))) &
                                                               ((otp_buste_itineraries_summary['start_diff'] > pd.Timedelta('0s')) & (otp_buste_itineraries_summary['start_diff'] < pd.Timedelta('1.5h')))] \
                                    .query('origin_dist < 0.1') \
                                    .query('next_origin_dist < 2.0')

In [90]:
otp_buste_itineraries_penalty = otp_buste_itineraries_filtered \
                                    .assign(penalty = lambda x: 2*x['start_diff'].dt.total_seconds() + x['trip_duration'].dt.total_seconds() + x['match_num_transfers']*10) \
                                    [['cardNum','user_trip_id','itinerary_id','match_num_transfers','match_vehicle_boarding','next_origin_dist','origin_dist','start_diff','trip_duration','penalty']] \
                                    .sort_values(['user_trip_id','penalty'], ascending=True)

In [91]:
otp_buste_itineraries_penalty.head()

,cardNum,user_trip_id,itinerary_id,match_num_transfers,match_vehicle_boarding,next_origin_dist,origin_dist,start_diff,trip_duration,penalty
0,568690.0,1496,2,4,True,0.026183,0.0,00:01:02,01:26:26,5350.0
1,568690.0,1496,4,4,True,0.838850,0.0,00:41:23,01:02:03,8729.0
2,618552.0,1869,2,3,True,0.000000,0.0,00:00:59,01:11:48,4456.0
3,618552.0,1869,4,3,True,0.000000,0.0,00:19:28,01:06:17,6343.0
4,618552.0,1869,5,4,True,0.189657,0.0,00:19:28,01:20:24,7200.0


In [92]:
chosen_itineraries = otp_buste_itineraries_penalty.groupby(['user_trip_id']).first().reset_index()

In [93]:
len(chosen_itineraries)

644

In [94]:
chosen_itineraries.head()

,user_trip_id,cardNum,itinerary_id,match_num_transfers,match_vehicle_boarding,next_origin_dist,origin_dist,start_diff,trip_duration,penalty
0,1496,568690.0,2,4,True,0.026183,0.0,00:01:02,01:26:26,5350.0
1,1869,618552.0,2,3,True,0.000000,0.0,00:00:59,01:11:48,4456.0
2,1870,618552.0,6,3,True,0.012721,0.0,00:11:02,00:36:18,3532.0
3,1947,623225.0,1,2,True,0.005337,0.0,00:00:24,00:36:15,2243.0
4,2028,635030.0,1,3,True,1.096995,0.0,00:00:51,00:40:58,2590.0


In [95]:
chosen_itineraries = chosen_itineraries[(np.logical_not(chosen_itineraries['match_vehicle_boarding'])) | ((chosen_itineraries['match_vehicle_boarding']) & (chosen_itineraries['start_diff'] < pd.Timedelta('20 min')))]

In [96]:
len(chosen_itineraries)

586

In [97]:
chosen_itineraries.describe(percentiles=[.25,.5,.75,.9,.95,.99])

,user_trip_id,cardNum,itinerary_id,match_num_transfers,next_origin_dist,origin_dist,start_diff,trip_duration,penalty
count,586.000000,5.860000e+02,586.000000,586.000000,586.000000,586.000000,586,586,586.000000
mean,116640.546075,2.661931e+06,1.733788,2.296928,0.171921,0.005894,0 days 00:06:36.515358,0 days 00:54:03.571672,4059.571672
std,81136.946512,9.031234e+05,1.325922,0.810544,0.281732,0.016516,0 days 00:11:53.775487,0 days 00:29:08.340393,1973.397693
min,1496.000000,5.686900e+05,1.000000,1.000000,0.000000,0.000000,0 days 00:00:01,0 days 00:00:14,204.000000
25%,46667.500000,2.030924e+06,1.000000,2.000000,0.008472,0.000000,0 days 00:00:19,0 days 00:28:11.500000,2590.250000
50%,102828.500000,2.746774e+06,1.000000,2.000000,0.034724,0.000000,0 days 00:00:59,0 days 00:54:18,4029.000000
75%,184419.750000,3.523690e+06,2.000000,3.000000,0.226465,0.001339,0 days 00:08:04.250000,0 days 01:15:22.750000,5232.000000
90%,239852.500000,3.747450e+06,3.000000,3.000000,0.559183,0.018954,0 days 00:18:38.500000,0 days 01:31:02.500000,6637.500000
95%,254917.250000,3.785365e+06,5.000000,4.000000,0.781840,0.043591,0 days 00:29:01.500000,0 days 01:45:59.750000,7641.000000
99%,273826.100000,3.822110e+06,7.000000,4.000000,1.385745,0.084842,0 days 00:57:55.799999,0 days 01:58:28.849999,9094.550000


In [98]:
chosen_itineraries.head()

,user_trip_id,cardNum,itinerary_id,match_num_transfers,match_vehicle_boarding,next_origin_dist,origin_dist,start_diff,trip_duration,penalty
0,1496,568690.0,2,4,True,0.026183,0.0,00:01:02,01:26:26,5350.0
1,1869,618552.0,2,3,True,0.000000,0.0,00:00:59,01:11:48,4456.0
2,1870,618552.0,6,3,True,0.012721,0.0,00:11:02,00:36:18,3532.0
3,1947,623225.0,1,2,True,0.005337,0.0,00:00:24,00:36:15,2243.0
4,2028,635030.0,1,3,True,1.096995,0.0,00:00:51,00:40:58,2590.0


In [99]:
chosen_itineraries.dtypes

user_trip_id                        int64
cardNum                           float64
itinerary_id                        int64
match_num_transfers                 int64
match_vehicle_boarding               bool
next_origin_dist                  float64
origin_dist                       float64
start_diff                timedelta64[ns]
trip_duration             timedelta64[ns]
penalty                           float64
dtype: object

In [100]:
otp_legs_buste_data.head()

,cardNum,user_trip_id,first_vehicle_boarding,itinerary_id,leg_id,route,busCode,o_busCode,tripNum,o_tripNum,...,to_stop_id,otp_end_time,matched_end_time,otp_buste_end_timediff,boarding_otp_match_start_timediff,leg_duration,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
0,568690.0,1496,True,2,1,226,BA140,BA140,1.0,1.0,...,30452,2017-05-09 05:20:00,2017-05-09 05:14:02,00:05:58,00:01:02,00:10:30,-25.373245,-49.244344,-25.393259,-49.240989
1,568690.0,1496,False,2,3,203,BD131,BA140,1.0,1.0,...,25389,2017-05-09 05:45:54,2017-05-09 05:47:45,00:01:51,00:21:34,00:21:37,-25.393517,-49.240616,-25.437593,-49.266727
2,568690.0,1496,False,2,4,303,DE701,BA140,1.0,1.0,...,26195,2017-05-09 06:20:00,2017-05-09 06:14:56,00:05:04,00:40:04,00:30:18,-25.437593,-49.266727,-25.441720,-49.346978
3,568690.0,1496,False,2,6,732,JA024,BA140,1.0,1.0,...,33185,2017-05-09 06:30:19,2017-05-09 06:29:58,00:00:21,01:14:08,00:11:16,-25.441531,-49.346711,-25.451980,-49.362003
4,568690.0,1496,True,4,1,226,BA140,BA140,2.0,1.0,...,30452,2017-05-09 05:44:00,2017-05-09 06:03:33,00:19:33,00:41:23,00:17:36,-25.373245,-49.244344,-25.393259,-49.240989


In [101]:
otp_legs_buste_data.dtypes

cardNum                                      float64
user_trip_id                                   int64
first_vehicle_boarding                          bool
itinerary_id                                   int64
leg_id                                         int64
route                                         object
busCode                                       object
o_busCode                                     object
tripNum                                      float64
o_tripNum                                    float64
from_stop_id                                  object
otp_start_time                        datetime64[ns]
matched_start_time                    datetime64[ns]
o_boarding_datetime                   datetime64[ns]
otp_buste_start_timediff             timedelta64[ns]
to_stop_id                                    object
otp_end_time                          datetime64[ns]
matched_end_time                      datetime64[ns]
otp_buste_end_timediff               timedelta

In [102]:
otp_legs_buste_data.head()

,cardNum,user_trip_id,first_vehicle_boarding,itinerary_id,leg_id,route,busCode,o_busCode,tripNum,o_tripNum,...,to_stop_id,otp_end_time,matched_end_time,otp_buste_end_timediff,boarding_otp_match_start_timediff,leg_duration,from_stop_lat,from_stop_lon,to_stop_lat,to_stop_lon
0,568690.0,1496,True,2,1,226,BA140,BA140,1.0,1.0,...,30452,2017-05-09 05:20:00,2017-05-09 05:14:02,00:05:58,00:01:02,00:10:30,-25.373245,-49.244344,-25.393259,-49.240989
1,568690.0,1496,False,2,3,203,BD131,BA140,1.0,1.0,...,25389,2017-05-09 05:45:54,2017-05-09 05:47:45,00:01:51,00:21:34,00:21:37,-25.393517,-49.240616,-25.437593,-49.266727
2,568690.0,1496,False,2,4,303,DE701,BA140,1.0,1.0,...,26195,2017-05-09 06:20:00,2017-05-09 06:14:56,00:05:04,00:40:04,00:30:18,-25.437593,-49.266727,-25.441720,-49.346978
3,568690.0,1496,False,2,6,732,JA024,BA140,1.0,1.0,...,33185,2017-05-09 06:30:19,2017-05-09 06:29:58,00:00:21,01:14:08,00:11:16,-25.441531,-49.346711,-25.451980,-49.362003
4,568690.0,1496,True,4,1,226,BA140,BA140,2.0,1.0,...,30452,2017-05-09 05:44:00,2017-05-09 06:03:33,00:19:33,00:41:23,00:17:36,-25.373245,-49.244344,-25.393259,-49.240989


In [103]:
od_trips = chosen_itineraries.merge(otp_legs_buste_data, on=['cardNum','user_trip_id','itinerary_id'], how='inner') \
        [['cardNum','user_trip_id','itinerary_id','leg_id','route','busCode','tripNum','from_stop_id','matched_start_time','from_stop_lat','from_stop_lon','to_stop_id','matched_end_time','to_stop_lat','to_stop_lon','leg_duration']] \
        .rename(index=str, columns={'matched_start_time':'start_time','matched_end_time':'end_time'})

In [104]:
od_trips.head()

,cardNum,user_trip_id,itinerary_id,leg_id,route,busCode,tripNum,from_stop_id,start_time,from_stop_lat,from_stop_lon,to_stop_id,end_time,to_stop_lat,to_stop_lon,leg_duration
0,568690.0,1496,2,1,226,BA140,1.0,34486,2017-05-09 05:03:32,-25.373245,-49.244344,30452,2017-05-09 05:14:02,-25.393259,-49.240989,00:10:30
1,568690.0,1496,2,3,203,BD131,1.0,26190,2017-05-09 05:26:08,-25.393517,-49.240616,25389,2017-05-09 05:47:45,-25.437593,-49.266727,00:21:37
2,568690.0,1496,2,4,303,DE701,1.0,25389,2017-05-09 05:44:38,-25.437593,-49.266727,26195,2017-05-09 06:14:56,-25.441720,-49.346978,00:30:18
3,568690.0,1496,2,6,732,JA024,1.0,31002,2017-05-09 06:18:42,-25.441531,-49.346711,33185,2017-05-09 06:29:58,-25.451980,-49.362003,00:11:16
4,618552.0,1869,2,1,653,HA270,1.0,36174,2017-05-09 05:02:18,-25.500684,-49.351664,30435,2017-05-09 05:17:17,-25.511621,-49.324356,00:14:59
